# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//afarbin/DATA1401-Spring-2020/blob/master/Labs/Lab-7/Lab-7.ipynb)

Here are the "Gradebook" classes from lecture. For this lab, you will use these classes and are encouraged to modify them as you need.

In [393]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [394]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Algorithm")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print (self.name()+" Error: Bad Grade.")
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])
    


In [395]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError


In [396]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
    
    def data(self):
        return self.__data
    
    def print_data(self):
        for k,v in self.__data.items():
            print (k,":",v)
    
    def print_students(self):
        for k,a_student in self.__students.items():
            print (k, a_student.name())
            a_student.print_grades()
            print ("_______________________________________")
    
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)
        
    

In [437]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

In [398]:
class mean_std_calculator(calculator):
    def __init__(self,grade_name):
        self.__grade_name = grade_name
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if not grade_name:
            grade_name = self.__grade_name
            
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))


## CSV Reader

*Exercise 1*: The data for a class are stored in a "camma separated values" (CSV) file name `Data1401-Grades.csv` in the directory of this lab. You can see the contents using the `cat` shell command:

In [399]:
!cat Data1401-Grades.csv 

l1_n,l1_1,l2_n,l2_1,l2_2,l2_3,l2_4,l2_5,l2_6,l2_7,l3_n,l3_1,l3_2,l3_3,l3_4,l3_5,l3_6,l3_7,l3_8,l3_9,l3_10,l3_11,l3_12,l3_13,l3_14,l4_n,l4_1,l4_2,l4_3,l4_4,l4_5,l4_6,l4_7,l4_8,l4_9,l4_10,l4_11,q1_n,q1_1,e1_n,e1_1,e1_2,e1_3,e1_4,e1_5,e1_6,e1_7,e1_8,e1_9,e1_10,e1_11,e1_12,e1_13,e1_14,e1_15
1,10,7,0,10,10,8,10,10,10,14,9,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,9.5,15,9,9,0,9,8,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,9,10,10,10,7,10,3,6,3,3,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,5,15,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0
1,10,7,10,10,3,9.5,10,10,9.5,14,10,10,10,8,5,10,5,10,3,0,10,3,10,8,11,10,10,10,10,10,10,0,0,10,5,0,1,10,15,9,9,10,9,7,9,0,0,10,10,9,5,10,8,10
1,10,7,10,10,9.5,0,10,10,0,14,9.5,0,0,10,0,10,5,10,7,0,10,6,10,0,11,10,10,6,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,5,0,7,0,3,3,3,0,3,0,0
1,10,7,10,10,10,9.5,10,10,9.5,14,5,9.5,9.5,8,10,10,8,10,8,0,5,6,0,0,11,0,10,10,10,0,5,0,

You will note that the first line has the names of the "columns" of data, and that subsequent lines (or "rows") have the data for each student, separated by cammas.

Recalling that in lecture we created a file reader, create a CSV reader function that takes a filename as input and returns data structure(s) that store the data in the file. Note that you are not allowed to use a library. The point here is for *you* to write the CSV reader. Some options for your data structures (pick one):

* A list of dictionaries, where each element of the list is corresponds to a row of data and the dictionaries are keyed by the column name. For example `data[5]["l3_5"]` corresponds to the 6th student's grade on lab 3 question 5.

* A list of lists (i.e. a 2-D array or matrix) and a dictionary, where each element of the "matrix" corresponds to a a specific grade for a specific student and the dictionary maps the name of the column to the column index. For example `data[5][column_names["l1_5"]]` corresponds to the 6th student's grade on lab 3 question 5.

* A dictionary of lists, where each element of the dictionary corresponds to a column of data and the lists contain the data in that column. For example `data["l3_5"][5]` corresponds to the 6th student's grade on lab 3 question 5.

* (Extra Credit) A class that simultaneously supports all of the above methods.

In [400]:
# Your solution here.

def cvs_reader(filename):
    data=list() # if you choose first option
    f=open(filename,"r")
    columns = f.readline().rstrip().split(",")
    
    for line in f:
        row = dict()
        l = line.rstrip()
        items = l.split(",")
        for i, item in enumerate(items):
            row[columns[i]] = item
        data.append(row)
    f.close()
    return data

cvs_reader("Data1401-Grades.csv")

[{'l1_n': '1',
  'l1_1': '10',
  'l2_n': '7',
  'l2_1': '0',
  'l2_2': '10',
  'l2_3': '10',
  'l2_4': '8',
  'l2_5': '10',
  'l2_6': '10',
  'l2_7': '10',
  'l3_n': '14',
  'l3_1': '9',
  'l3_2': '0',
  'l3_3': '0',
  'l3_4': '0',
  'l3_5': '0',
  'l3_6': '0',
  'l3_7': '0',
  'l3_8': '0',
  'l3_9': '0',
  'l3_10': '0',
  'l3_11': '0',
  'l3_12': '0',
  'l3_13': '0',
  'l3_14': '0',
  'l4_n': '11',
  'l4_1': '0',
  'l4_2': '0',
  'l4_3': '0',
  'l4_4': '0',
  'l4_5': '0',
  'l4_6': '0',
  'l4_7': '0',
  'l4_8': '0',
  'l4_9': '0',
  'l4_10': '0',
  'l4_11': '0',
  'q1_n': '1',
  'q1_1': '9.5',
  'e1_n': '15',
  'e1_1': '9',
  'e1_2': '9',
  'e1_3': '0',
  'e1_4': '9',
  'e1_5': '8',
  'e1_6': '0',
  'e1_7': '0',
  'e1_8': '0',
  'e1_9': '0',
  'e1_10': '0',
  'e1_11': '0',
  'e1_12': '0',
  'e1_13': '0',
  'e1_14': '0',
  'e1_15': '0'},
 {'l1_n': '1',
  'l1_1': '10',
  'l2_n': '7',
  'l2_1': '0',
  'l2_2': '0',
  'l2_3': '0',
  'l2_4': '0',
  'l2_5': '0',
  'l2_6': '0',
  'l2_7': '0',

## Creating a Gradebook

*Exercise 2:* In lecture we used pandas to read the CSV file and create the grade book. The example below works for the CSV file for this lab. Modify the code below to use your CSV reader instead.

In [401]:
class_data=cvs_reader("Data1401-Grades.csv")

a_grade_book=grade_book("Data 1401")

for student_i in range(len(class_data)):
    a_student_0=student("Student",str(student_i),student_i)

    for k in class_data[student_i].keys():
        a_student_0.add_grade(grade(k,value=float(class_data[student_i][k])))

    a_grade_book.add_student(a_student_0)
        

## Grade Summing

*Exercise 3:* In lectre we will change the design of our algorithm classes and then update the `uncurved_letter_grade_percent` calculator. In lecture we also created a `grade_summer` calcuator that takes a prefix (for example `e1_` and a number `n`) and sums all grades starting with that prefix up to `n` and creates a new sum grade. Update this calculator (below) to the new design of our algorithm classes. Test your updated calculator by using it to sum the grades for all labs, quizzes, and exams of each student.

In [402]:
# Note this is the OLD design... you will need to modify it.

class summary_calculator(alg):   
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_student):
        raise NotImplementedError

class grade_summer(calculator):
    def __init__(self,prefix,n):
        self.__prefix=prefix
        self.__n=n
        calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_gradebook,**kwargs):
        labels=[self.__prefix + str(x) for x in range(1,self.__n)]
        
        for k,a_student in a_grade_book.get_students().items():            

            grade_sum=0.
            for label in labels:
                grade_sum+=a_student[label].value()

            a_student.add_grade(grade(self.__prefix+"sum",value=grade_sum),**kwargs)

In [403]:
a_grade_book=grade_book("Data 1401")

for student_i in range(len(class_data)):
    a_student_0=student("Student",str(student_i),student_i)

    for k in class_data[student_i].keys():
        a_student_0.add_grade(grade(k,value=float(class_data[student_i][k])))

    a_grade_book.add_student(a_student_0)
#print(class_data)
prefixes=sorted(list(set([k.split("_")[0] + "_" for k in class_data[0].keys()])))
#labels=[self.__prefix+str(x) for x in range(1,self.__n)]
print(prefixes)
for j, prefix in enumerate(prefixes):
    a_grade_book.apply_calculator(grade_summer(prefix, int(list(a_grade_book.get_students().values())[j][prefix+"n"].value())+1))

['e1_', 'l1_', 'l2_', 'l3_', 'l4_', 'q1_']


In [404]:
for prefix in prefixes:
    print(prefix,list(a_student[prefix+"sum"].value() for k,a_student in a_grade_book.get_students().items()))

e1_ [35.0, 0, 20.0, 115.0, 24.0, 95.0, 37.0, 77.0, 103.0, 117.0, 138.0, 110.0, 104.0, 111.0, 103.0, 134.0]
l1_ [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
l2_ [58.0, 0, 0, 62.0, 49.5, 69.0, 54.5, 69.0, 39.5, 60.0, 47.0, 68.5, 64.0, 49.5, 68.0, 59.0]
l3_ [9.0, 0, 71.0, 102.0, 77.5, 89.0, 64.5, 40.0, 40.0, 26.0, 126.0, 97.5, 115.0, 114.5, 95.0, 106.0]
l4_ [0, 0, 0, 75.0, 26.0, 35.0, 30.0, 51.0, 68.0, 27.0, 105.0, 51.0, 0, 25.0, 0, 53.0]
q1_ [9.5, 0, 5.0, 10.0, 0, 9.5, 10.0, 10.0, 9.5, 9.5, 0, 10.0, 10.0, 0, 0, 9.5]


# Curving Grades

*Exercise 4:* Use the `mean_std_calculator` above to calculate the mean and standard deviation for every lab, quiz, and exam in the class. Add a new print function to the `grade_book` class to print out such information in a nice way, and use this function to show your results.


In [405]:
# Your solution here
prefixes=sorted(list(set([k.split("_")[0] + "_" for k in class_data[0].keys()])))
for prefix in prefixes:
    a_grade_book.apply_calculator(mean_std_calculator(grade_name=prefix+"sum"))
a_grade_book.print_data()

e1_sum Mean : 82.6875
e1_sum STD : 42.937045121316864
l1_sum Mean : 10.0
l1_sum STD : 0.0
l2_sum Mean : 51.09375
l2_sum STD : 21.05663401252679
l3_sum Mean : 73.3125
l3_sum STD : 38.301792957379945
l4_sum Mean : 34.125
l4_sum STD : 30.421774685248064
q1_sum Mean : 6.40625
q1_sum STD : 4.469405546322688


*Exercise 5:* In lecture we will change the design of our algorithms classes and then update the `uncurved_letter_grade_percent` calculator. Do the same for the `curved_letter_grade` calculator below and by curving all the lab, quiz, and exam grades.

In [428]:
class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
    
        for k,a_student in a_grade_book.get_students().items():
            a_grade = a_student[grade_name]

            if not a_grade.numerical():
                print(self.name()+"Error: Did not get a numerical grade as input.")
                raise Exception

            # Rescale the grade
            scaled_percent=1
            if self.__std != 0:
                percent=a_grade.value()/self.__max_grade
                shift_to_zero=percent-(self.__mean/self.__max_grade)
                scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
                scaled_percent=scale_std+0.8

            for i,v in enumerate(self.__grades_definition):
                if scaled_percent>=v[0]:
                    break

            overwrite = kwargs["overwrite"] if "overwrite" in kwargs else False
            #g = grade(grade_name+" Letter",value=self.__grades_definition[i][1])
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]),overwrite=overwrite)


In [436]:
for prefix in prefixes:
    grade_name = prefix+"sum"
    a_grade_book.apply_calculator(uncurved_letter_grade_percent(grade_name,max_grade=100))

AttributeError: 'dict' object has no attribute 'iteritems'

In [431]:
for prefix in prefixes:
    grade_name = prefix+"sum"
    a_grade_book.apply_calculator(curved_letter_grade(grade_name,
                                                      a_grade_book[grade_name+" Mean"],
                                                      a_grade_book[grade_name+" STD"]),
                                  overwrite=True)
a_grade_book.print_students()

0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
l2_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
e1_sum: 35.0
l1_sum: 10.0
l2_sum: 58.0
l3_sum: 9.0
l4_sum: 0
q1_sum: 9.5
e1_sum Letter: C+
l1_sum Letter: A+
l2_sum Letter: B
l3_sum Letter: C
l4_sum Letter: C+
q1_sum Letter: B
_______________________________________
1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
l2_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 

## Final Course Grade

*Exercise 6:* Write a new calculator that sums grades with a prefix, as in the `grade_summer` calculator, but drops `n` lowest grades. Apply the algorithm to drop the lowest lab grade in the data.


In [ ]:
# Your solution here

*Exercise 7*: Write a new calculator that creates a new letter grade based on a weighted average of letter grades, by assigning the following numerical values to letter grades:

In [ ]:
GradeMap={"A+":12,
          "A":11,
          "A-":10,
          "B+":9,
          "B":8,
          "B-":7,
          "C+":6,
          "C":5,
          "C-":4,
          "D+":3,
          "D":2,
          "D-":1,
          "F":0}

Test you calculator by applying the weights from the syllabus of this course and computing everyone's grade in the course.

In [ ]:
# Your solution here